In [ ]:
import plotly.graph_objects as go
import pandas as pd
from pandas import DataFrame as df
import numpy as np
import plotly.express as px
from EC_data_processing_lib import get_header_length
from EC_data_processing_lib import analyse_file
from EC_data_processing_lib import get_decimal_separator
from Data_setup_TMNML import data_set
import os
from galvani import BioLogic
fig_title = 'TMNS first paper'
fig = go.Figure()
colors = px.colors.qualitative.Dark24

for i in data_set:
    if not os.path.exists(data_set[i]['data_CV']):
        print("File not found: " + file)
        continue
    if data_set[i]['data_CV'].endswith('mpt'):
        headerlength = get_header_length(data_set[i]['data_CV'])
        data = pd.read_csv(data_set[i]['data_CV'], encoding='ansi', sep="\t", decimal=get_decimal_separator(data_set[i]['data_CV']),  skiprows=range(headerlength))
        current_column = '<I>/mA'
    else:
        mpr_file = BioLogic.MPRfile(data_set[i]['data_CV'])
        data = pd.DataFrame(mpr_file.data)
        current_column = 'I/mA'
    cn = data['cycle number']
    #data = data[data['cycle number']== float(cn[-1:])] # allowing this comment will plot only the last scan. If you want to plot all of the scans then comment this out. 
    voltage = data['Ewe/V']
    current = data[current_column]
    current = current / data_set[i]['mass']
    voltage_corrected = data['Ewe/V'] - data[current_column] * data_set[i]['Ru']/1000 - data_set[i]['cal'] #does the iR correction converts to overpotential
    
    
    j = 0
    max_current = []
    for j in range(int(cn.tail(1))):
        max_current.append(df.max(current.loc[data['cycle number']==j+1]))
        
    print(data_set[i]['label'], max_current[0], max_current[2])

    fig.add_trace(go.Scatter(
        x=voltage_corrected,
        y=current,
        name= data_set[i]['label'],
        line_color = colors[data_set[i]['color_index']],
    ))
fig.update_xaxes(range=[0.00, 0.750],
    nticks=10,
    ticks='outside',
    showgrid=False, gridwidth=1, gridcolor='grey',
    zeroline=False, zerolinecolor='grey', zerolinewidth= 1,
    showline=True, linewidth=2, linecolor='black')

fig.update_yaxes(range=[-5, 160],
    nticks=10,
    showgrid=False, gridwidth=1, gridcolor='grey',
    ticks='outside',
    zeroline=True, zerolinecolor='grey', zerolinewidth= 1,
    showline=True, linewidth=2, linecolor='black')
fig.update_layout(
    #title="CV",
    
    xaxis_title="Overpotential, η(V) - iR<sub>u</sub>",
    yaxis_title="Current (mA/cm<sup>2</sup>)",
    font=dict(
        family="Times New Roman, monospace",
        size=22,
        color="dimgrey"
        
    )
)

fig.update_layout({
    "plot_bgcolor": "rgba(0, 0, 0, 0)",
    'paper_bgcolor': 'rgba(255, 255, 255, 255)', #sets the background to be transparent... might appear black in .jpeg but will be transparent in the .svg
},
width = 1000,
height = 500,   
legend=dict(
    yanchor="top",
    y=1,
    xanchor="left",
    x=1
),
margin={'t':0,'l':0,'b':0,'r':0},
legend_font_size=10)

      

fig.show()

fig.write_image(fig_title +' Cyclic Voltammetry.jpeg')
fig.write_image(fig_title + ' Cyclic Voltammetry.svg')

